# 영화 랭킹 정보 조회 및 포스터 이미지 저장하기

In [5]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os

In [6]:
movie_ranking = requests.get("https://www.moviechart.co.kr/rank/realtime/index/image")

image_dir = 'images'
if not os.path.exists(image_dir):
  os.makedirs(image_dir)

pattern = r'[\\/:"*?<>|]' #파일이나 폴더 이름으로 절대 사용할 수 없는 특수문자 9가지

if movie_ranking.status_code == 200:
  print("영화 정보를 출력합니다.")
  soup = BeautifulSoup(movie_ranking.content, 'html.parser')
  
  #content > div.wArea.space > div.movieBox > ul > li:nth-child(2) > div > div.movie-title > h3 > a
  movie_title_list = soup.select(".movieBox-list .movie-title a") # 영화 이름 <a> 요소 목록
  movie_image_list = soup.select(".movieBox-list .movieBox-item img") #img 태그 요소
  print(f"수집한 영화 수: {len(movie_title_list)}")

  for movie_title, movie_image in zip(movie_title_list, movie_image_list):
    print(movie_title.text, movie_image.get('src'))  #이름 요소의 텍스트, 이미지 요소의 src속성의 값
    
    image_src = movie_image.get('src')
    image_response = requests.get("https://www.moviechart.co.kr" + image_src)
    img = Image.open(BytesIO(image_response.content))
    image_filename = re.sub(pattern, '', movie_title.text) #파일이름 지정, 정규식
    img.save(os.path.join(image_dir, image_filename + ".png"))
    print(movie_title.text, )  
else:
  print("페이지에 연결할 수 없습니다.")

영화 정보를 출력합니다.
수집한 영화 수: 20
왕과 사는 남자 /thumb?width=178&height=267&m_code=20242837&source=https://admin.moviechart.co.kr/assets/upload/movie/260108003526_8322.jpg
왕과 사는 남자
휴민트 /thumb?width=178&height=267&m_code=20241266&source=https://admin.moviechart.co.kr/assets/upload/movie/260112024651_6301.jpg
휴민트
초속 5센티미터 /thumb?width=178&height=267&m_code=20259583&source=https://admin.moviechart.co.kr/assets/upload/movie/260213052538_7190.jpg
초속 5센티미터
너자 2 /thumb?width=178&height=267&m_code=20261181&source=https://admin.moviechart.co.kr/assets/upload/movie/260202063756_4036.jpg
너자 2
슬라이드 스트럼 뮤트 /thumb?width=178&height=267&m_code=20261450&source=https://admin.moviechart.co.kr/assets/upload/movie/260219042530_2928.jpg
슬라이드 스트럼 뮤트
넘버원 /thumb?width=178&height=267&m_code=20252373&source=https://admin.moviechart.co.kr/assets/upload/movie/260123065552_5690.jpg
넘버원
햄넷 /thumb?width=178&height=267&m_code=20250365&source=https://admin.moviechart.co.kr/assets/upload/movie/260202063420_3704.jpg
햄넷
부흥 /thumb?wid

# 수정 버전

In [1]:
import os, re
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
from urllib.parse import urljoin

In [7]:
URL = "https://www.moviechart.co.kr/rank/realtime/index/image"
BASE = "https://www.moviechart.co.kr"

In [ ]:
image_dir = "images"
os.makedirs(image_dir, exist_ok=True)

pattern = r'[\\/:"*?<>|]'

headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(URL, headers=headers, timeout=10)
resp.raise_for_status()

soup = BeautifulSoup(resp.content, "html.parser")

titles = soup.select(".movieBox-list .movie-title a")
imgs = soup.select(".movieBox-list .movieBox-item img")

print(f"수집한 영화 수: {min(len(titles), len(imgs))}")

for idx, (t, imgtag) in enumerate(zip(titles, imgs), start=1):
    title = t.get_text(strip=True)
    src = imgtag.get("src")

    if not src:
        print(f"[{idx}] 이미지 src 없음: {title}")
        continue

    img_url = urljoin(BASE, src)

    img_resp = requests.get(img_url, headers=headers, timeout=10)
    if img_resp.status_code != 200:
        print(f"[{idx}] 이미지 다운로드 실패({img_resp.status_code}): {title}")
        continue

    try:
        img = Image.open(BytesIO(img_resp.content))
    except Exception as e:
        print(f"[{idx}] 이미지 열기 실패: {title} / {e}")
        continue

    safe_name = re.sub(pattern, "", title).strip()
    filename = f"{idx:02d}_{safe_name}.png"  # 순위 prefix로 중복 방지
    img.save(os.path.join(image_dir, filename))
    print(f"[{idx}] 저장 완료: {filename}")

# 영화 포스터 수집 예에서 포스터 원본을 저장

In [2]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os
from urllib.parse import urlparse, parse_qs

In [3]:
movie_ranking = requests.get("https://www.moviechart.co.kr/rank/realtime/index/image")

image_dir = 'images2'
if not os.path.exists(image_dir):
  os.makedirs(image_dir)

pattern = r'[\\/:"*?<>|]' 

if movie_ranking.status_code == 200:
  print("영화 정보를 출력합니다.")
  soup = BeautifulSoup(movie_ranking.content, 'html.parser')
  movie_title_list = soup.select(".movieBox-list .movie-title a")
  movie_image_list = soup.select(".movieBox-list .movieBox-item img")
  print(f"수집한 영화 수: {len(movie_title_list)}")

  for movie_title, movie_image in zip(movie_title_list, movie_image_list):
    url = movie_image.get('src')
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    image_src = query_params.get('source', [None])[0]
    # image_response = requests.get('https://www.moviechart.co.kr' + image_src)
    image_response = requests.get(image_src)
    img = Image.open(BytesIO(image_response.content))
    image_filename = re.sub(pattern, '', movie_title.text)
    img.save(os.path.join(image_dir, image_filename + '.png'))
    print(movie_title.text, )
else:
  print("페이지에 연결할 수 없습니다.")

영화 정보를 출력합니다.
수집한 영화 수: 20
왕과 사는 남자
휴민트
초속 5센티미터
너자 2
슬라이드 스트럼 뮤트
넘버원
햄넷
부흥
매드 댄스 오피스
신의악단
렌탈 패밀리: 가족을 빌려드립니다
몬테크리스토 백작
호퍼스
만약에 우리
프랑켄슈타인
폭풍의 언덕
점보
아기 티라노 디보: 초식이지만 괜찮아!
직사각형, 삼각형
아바타: 불과 재
